# Build Central Lake Ontario Buoy Dataset
---
### Carter J. Humphreys
Email: [chumphre@oswego.edu](mailto:chumphre@oswego.edu) | GitHub:[@HumphreysCarter](https://github.com/HumphreysCarter) | Website: [carterhumphreys.com](http://carterhumphreys.com/)

In [224]:
import pandas as pd
import numpy as np
import math

In [225]:
dataDir='../data'

# Extract LO1 Data

In [226]:
dataset=pd.read_csv(f'{dataDir}/full_dataset.csv', header=None)
dataset=dataset.drop(range(34, 334), axis=1)
dataset=dataset.rename(columns={0:'DateTime [UTC]', 1:'BandStart_Latitude', 2:'BandStart_Longitude', 3:'BandMidpoint_Latitude', 4:'BandMidpoint_Longitude', 5:'BandEnd_Latitude', 6:'BandEnd_Longitude', 7:'BandIntensity [dBZ]', 8:'BandAz_LO1 [deg]', 9:'BandRng_LO1 [km]', 10:'WaterTemp_Ontario [degC]', 11:'IceCover_Ontario [%]', 12:'IceCover_Huron [%]', 13:'IceCover_Erie [%]', 14:'z_925mb [m]', 15:'T_925mb [degC]', 16:'RH_925mb [%]', 17:'u_925mb [kt]', 18:'v_925mb [kt]', 19:'z_850mb [m]', 20:'T_850mb [degC]', 21:'RH_850mb [%]', 22:'u_850mb [kt]', 23:'v_850mb [kt]', 24:'z_700mb [m]', 25:'T_700mb [degC]', 26:'RH_700mb [%]', 27:'u_700mb [kt]', 28:'v_700mb [kt]', 29:'z_500mb [m]', 30:'T_500mb [degC]', 31:'RH_500mb [%]', 32:'u_500mb [kt]', 33:'v_500mb [kt]'})
dataset

,DateTime [UTC],BandStart_Latitude,BandStart_Longitude,BandMidpoint_Latitude,BandMidpoint_Longitude,BandEnd_Latitude,BandEnd_Longitude,BandIntensity [dBZ],BandAz_LO1 [deg],BandRng_LO1 [km],...,z_700mb [m],T_700mb [degC],RH_700mb [%],u_700mb [kt],v_700mb [kt],z_500mb [m],T_500mb [degC],RH_500mb [%],u_500mb [kt],v_500mb [kt]
0,2015-10-17 10:00:00,43.535946,-76.223320,43.353896,-75.329963,43.234745,-74.595901,-9999.0,95.175078,95.981749,...,2961.09,-14.01,43.55,23.59,-9.42,5437.26,-30.38,27.49,37.65,-12.40
1,2015-10-17 11:00:00,43.532036,-76.217194,43.334646,-75.373511,43.143089,-74.490379,4.0,95.400996,96.518679,...,2962.97,-14.46,33.73,22.43,-11.62,5438.25,-29.61,25.61,36.14,-13.87
2,2015-10-17 12:00:00,43.521149,-76.232010,43.346326,-75.618544,43.195889,-74.990135,1.5,96.200389,95.472330,...,2964.81,-15.17,37.17,22.20,-12.70,5432.29,-30.14,19.75,34.44,-13.16
3,2015-10-17 13:00:00,43.518513,-76.217072,43.339364,-75.832468,43.176296,-75.413177,5.0,96.285512,96.702867,...,2970.15,-15.64,55.30,23.37,-11.55,5436.76,-30.23,16.43,35.66,-11.93
4,2015-10-17 14:00:00,43.565903,-76.202736,43.259891,-75.604233,42.989628,-75.002983,6.5,93.124002,97.344800,...,2970.41,-15.94,64.36,23.26,-12.20,5440.28,-29.95,15.63,35.60,-10.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1140,2019-04-01 06:00:00,43.302155,-76.797066,43.181426,-75.945325,42.964691,-75.063576,17.0,125.332050,60.756589,...,2865.34,-20.98,77.02,26.54,-5.38,5282.75,-33.25,7.75,27.71,17.99
1141,2019-04-01 07:00:00,43.548927,-76.197563,43.250276,-75.728180,42.978962,-75.235283,8.0,94.208553,97.908125,...,2870.55,-21.30,75.38,26.59,-7.02,5280.96,-34.34,8.66,29.66,14.71
1142,2019-04-01 08:00:00,43.553471,-76.185211,43.228888,-75.635044,42.981030,-75.192307,8.5,93.866284,98.856365,...,2864.63,-21.62,73.95,26.95,-7.65,5268.51,-35.26,11.34,30.79,9.28
1143,2019-04-01 09:00:00,43.320927,-76.710991,43.121541,-76.320671,42.869659,-75.870171,12.5,120.280638,65.436882,...,2865.98,-21.98,73.24,29.68,-4.40,5267.34,-35.68,10.89,32.32,5.90


# Derived Quantities

In [227]:
def calcWindShear(uUpr, uLwr, vUpr, vLwr):
    u_shear=uUpr-uLwr
    v_shear=vUpr-uLwr
    bulk_shear=math.sqrt(math.pow(v_shear, 2)+math.pow(v_shear, 2))
    bulk_shear=round(bulk_shear, 2)
    return {'u_shr':u_shear, 'v_shear':v_shear, 'bulk_shear':bulk_shear}

In [228]:
def calcLapseRate(T2, Z2, T1, Z1=0.0):
    dT=T2-T1
    dz=Z2-Z1
    return (dT/dz)*-1

In [229]:
# 925-850hPa, 925-700hPa, and 925-500hPa Wind Shear
dataset.insert(24, 'bulkshear_925-850hPa [kt]', np.nan)
dataset.insert(30, 'bulkshear_925-700hPa [kt]', np.nan)
dataset.insert(36, 'bulkshear_925-500hPa [kt]', np.nan)

u_925=dataset['u_925mb [kt]'].tolist()
v_925=dataset['v_925mb [kt]'].tolist()
u_850=dataset['u_850mb [kt]'].tolist()
v_850=dataset['v_850mb [kt]'].tolist()
u_700=dataset['u_700mb [kt]'].tolist()
v_700=dataset['v_700mb [kt]'].tolist()
u_500=dataset['u_500mb [kt]'].tolist()
v_500=dataset['v_500mb [kt]'].tolist()

# T_water-T_925hPa, T_water-T_850hPa, and T_water-T_700hPa lapse-rate
dataset.insert(16, 'dT/dz_water-925hPa [degC/km]', np.nan)
dataset.insert(22, 'dT/dz_water-850hPa [degC/km]', np.nan)
dataset.insert(29, 'dT/dz_water-700hPa [degC/km]', np.nan)

z_925=dataset['z_925mb [m]'].tolist()
z_850=dataset['z_850mb [m]'].tolist()
z_700=dataset['z_700mb [m]'].tolist()
T_wtr=dataset['WaterTemp_Ontario [degC]'].tolist()
T_925=dataset['T_925mb [degC]'].tolist()
T_850=dataset['T_850mb [degC]'].tolist()
T_700=dataset['T_700mb [degC]'].tolist()

index = 0
for i, row in dataset.iterrows():
    
    # 925-850hPa, 925-700hPa, and 925-500hPa Wind Shear
    shear_925_850=calcWindShear(u_850[index], u_925[index], v_850[index], v_925[index])
    shear_925_700=calcWindShear(u_700[index], u_925[index], v_700[index], v_925[index])
    shear_925_500=calcWindShear(u_500[index], u_925[index], v_500[index], v_925[index]) 
    dataset.iloc[index, dataset.columns.get_loc('bulkshear_925-850hPa [kt]')] = shear_925_850['bulk_shear']
    dataset.iloc[index, dataset.columns.get_loc('bulkshear_925-700hPa [kt]')] = shear_925_700['bulk_shear']
    dataset.iloc[index, dataset.columns.get_loc('bulkshear_925-500hPa [kt]')] = shear_925_500['bulk_shear'] 
    
    # T_water-T_925hPa, T_water-T_850hPa, and T_water-T_700hPa lapse-rate
    lapseRate_water_925 = calcLapseRate(T_925[index], z_925[index]/1000.0, T_wtr[index], 0.0)
    lapseRate_water_850 = calcLapseRate(T_925[index], z_850[index]/1000.0, T_wtr[index], 0.0)
    lapseRate_water_700 = calcLapseRate(T_925[index], z_700[index]/1000.0, T_wtr[index], 0.0)
    dataset.iloc[index, dataset.columns.get_loc('dT/dz_water-925hPa [degC/km]')] = lapseRate_water_925
    dataset.iloc[index, dataset.columns.get_loc('dT/dz_water-850hPa [degC/km]')] = lapseRate_water_850
    dataset.iloc[index, dataset.columns.get_loc('dT/dz_water-700hPa [degC/km]')] = lapseRate_water_700 
    
    # Go to next index
    index+=1

# Export Dataset

In [230]:
dataset.to_csv(f'{dataDir}/LO1_dataset.csv', header=True, index=False)
dataset

,DateTime [UTC],BandStart_Latitude,BandStart_Longitude,BandMidpoint_Latitude,BandMidpoint_Longitude,BandEnd_Latitude,BandEnd_Longitude,BandIntensity [dBZ],BandAz_LO1 [deg],BandRng_LO1 [km],...,RH_700mb [%],u_700mb [kt],v_700mb [kt],bulkshear_925-700hPa [kt],z_500mb [m],T_500mb [degC],RH_500mb [%],u_500mb [kt],v_500mb [kt],bulkshear_925-500hPa [kt]
0,2015-10-17 10:00:00,43.535946,-76.223320,43.353896,-75.329963,43.234745,-74.595901,-9999.0,95.175078,95.981749,...,43.55,23.59,-9.42,22.64,5437.26,-30.38,27.49,37.65,-12.40,26.86
1,2015-10-17 11:00:00,43.532036,-76.217194,43.334646,-75.373511,43.143089,-74.490379,4.0,95.400996,96.518679,...,33.73,22.43,-11.62,26.97,5438.25,-29.61,25.61,36.14,-13.87,30.15
2,2015-10-17 12:00:00,43.521149,-76.232010,43.346326,-75.618544,43.195889,-74.990135,1.5,96.200389,95.472330,...,37.17,22.20,-12.70,28.10,5432.29,-30.14,19.75,34.44,-13.16,28.75
3,2015-10-17 13:00:00,43.518513,-76.217072,43.339364,-75.832468,43.176296,-75.413177,5.0,96.285512,96.702867,...,55.30,23.37,-11.55,27.01,5436.76,-30.23,16.43,35.66,-11.93,27.55
4,2015-10-17 14:00:00,43.565903,-76.202736,43.259891,-75.604233,42.989628,-75.002983,6.5,93.124002,97.344800,...,64.36,23.26,-12.20,26.52,5440.28,-29.95,15.63,35.60,-10.08,23.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1140,2019-04-01 06:00:00,43.302155,-76.797066,43.181426,-75.945325,42.964691,-75.063576,17.0,125.332050,60.756589,...,77.02,26.54,-5.38,40.72,5282.75,-33.25,7.75,27.71,17.99,7.67
1141,2019-04-01 07:00:00,43.548927,-76.197563,43.250276,-75.728180,42.978962,-75.235283,8.0,94.208553,97.908125,...,75.38,26.59,-7.02,45.41,5280.96,-34.34,8.66,29.66,14.71,14.68
1142,2019-04-01 08:00:00,43.553471,-76.185211,43.228888,-75.635044,42.981030,-75.192307,8.5,93.866284,98.856365,...,73.95,26.95,-7.65,46.10,5268.51,-35.26,11.34,30.79,9.28,22.16
1143,2019-04-01 09:00:00,43.320927,-76.710991,43.121541,-76.320671,42.869659,-75.870171,12.5,120.280638,65.436882,...,73.24,29.68,-4.40,39.71,5267.34,-35.68,10.89,32.32,5.90,25.14


# Dataset Statistics

In [231]:
stats=dataset.describe()
stats.to_csv(f'{dataDir}/LO1_dataset_statistics.csv', header=True)
stats

,BandStart_Latitude,BandStart_Longitude,BandMidpoint_Latitude,BandMidpoint_Longitude,BandEnd_Latitude,BandEnd_Longitude,BandIntensity [dBZ],BandAz_LO1 [deg],BandRng_LO1 [km],WaterTemp_Ontario [degC],...,RH_700mb [%],u_700mb [kt],v_700mb [kt],bulkshear_925-700hPa [kt],z_500mb [m],T_500mb [degC],RH_500mb [%],u_500mb [kt],v_500mb [kt],bulkshear_925-500hPa [kt]
count,1145.000000,1145.000000,1145.000000,1145.000000,1145.000000,1145.000000,1145.000000,1145.000000,1145.000000,1145.000000,...,1145.000000,1145.000000,1145.000000,1145.000000,1145.000000,1145.000000,1145.000000,1145.000000,1145.000000,1145.000000
mean,43.630653,-76.381776,43.633758,-75.787178,43.639705,-75.142739,-463.378603,93.641756,87.640164,4.608821,...,56.317799,32.223721,6.645485,25.328533,5260.147886,-33.079852,34.751860,46.911380,13.336288,33.283066
std,0.233597,0.279259,0.348498,0.408128,0.519221,0.670774,2142.942481,22.816799,18.150216,2.408046,...,23.846803,10.059089,14.390203,18.191756,101.036994,5.168124,21.012772,20.137452,25.971267,23.508466
min,43.243790,-77.509819,42.907711,-77.060097,42.608696,-76.938240,-9999.000000,48.341748,38.160649,1.350000,...,0.970000,7.930000,-33.690000,0.010000,5006.450000,-45.880000,2.260000,-11.000000,-68.130000,0.040000
25%,43.446945,-76.559319,43.339364,-76.035680,43.210049,-75.653412,9.500000,78.997491,71.131557,2.820000,...,36.800000,25.100000,-3.340000,9.670000,5186.260000,-36.770000,16.630000,33.390000,-4.500000,15.150000
50%,43.617672,-76.225258,43.619020,-75.743159,43.589722,-75.121086,18.000000,89.729152,96.965336,3.970000,...,62.770000,31.220000,7.020000,22.360000,5263.640000,-33.150000,30.990000,45.340000,13.650000,29.920000
75%,43.780396,-76.199753,43.901171,-75.471099,44.032555,-74.616837,24.500000,105.391728,97.859283,6.140000,...,76.850000,38.390000,17.640000,38.040000,5321.090000,-28.890000,52.050000,60.680000,31.990000,47.450000
max,44.314522,-76.050560,44.518521,-74.867294,44.987915,-73.680901,41.500000,190.940376,130.345075,13.260000,...,99.130000,65.960000,45.270000,85.010000,5560.780000,-21.190000,98.090000,105.660000,83.550000,132.820000
